
Данная программа решает тест по бжд автоматически. Работает как на финальном, так и на тренировочных тестах.

Author: Maxim D.

## Требования

- Установленный google chrome
- Локальная среда jupiter

## Дисклеймер

Программа будет решать тест исходя из данных собранных в таблице. Таблица содержит некоторое количество неправильных ответов, поэтому в программу встроено несколько проверок ответов на адекватность. Если есть подозрения что ответ неправильный программа пропустит этот вопрос.

Проверки ответов, а также данные в таблице не 100% достоверны, поэтому могуть быть ошибки в 1-2% случаев.

## Инструкция

1. Запустите setup. Все необходимые библиотеки загрузятся и откроется новое окно chrome. Войдите в систему строго **в текущей вкладке**.
2. Запустите блок Parse answers from ans.csv. Ответы загрузятся в память программы
3. В той же вкладке, которую открыла программа, откройте и начните попытку в любом тесте. Если открыть тест в другой вкладке или окне, программа не будет иметь доступа к странице.
4. Запустите блок solve. Тест решится автоматически.
5. В ручную решите не отвеченные вопросы. При решении может помочь вывод программы.
6. (Опционально) Откройте другой тест и запустите solve еще раз



# Setup

In [2]:
%pip install chromedriver-autoinstaller selenium

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://lms.mipt.ru/")

Note: you may need to restart the kernel to use updated packages.


SessionNotCreatedException: Message: session not created: DevToolsActivePort file doesn't exist
Stacktrace:
#0 0x577f900ba8fa <unknown>
#1 0x577f8fbcbd20 <unknown>
#2 0x577f8fc08664 <unknown>
#3 0x577f8fc039fb <unknown>
#4 0x577f8fbfef96 <unknown>
#5 0x577f8fc4ad86 <unknown>
#6 0x577f8fc4a3d6 <unknown>
#7 0x577f8fc3e893 <unknown>
#8 0x577f8fc0d30d <unknown>
#9 0x577f8fc0e32e <unknown>
#10 0x577f9008700b <unknown>
#11 0x577f9008af97 <unknown>
#12 0x577f9007371c <unknown>
#13 0x577f9008bb17 <unknown>
#14 0x577f900586cf <unknown>
#15 0x577f900a96b8 <unknown>
#16 0x577f900a9880 <unknown>
#17 0x577f900b9776 <unknown>
#18 0x7f63dd49ca94 <unknown>
#19 0x7f63dd529c3c <unknown>


# Parse answers from ans.csv

In [ ]:
import csv
from collections import defaultdict

data_dict = defaultdict(list)
data_dict_all = defaultdict(list)

def is_english_letter(char):
    return 'a' <= char <= 'z'

# Trims a. b. c. d. from text beginning
def myTrim(value):
    if (len(value) > 3 and is_english_letter(value[0]) and value[1] == '.' and value[2] == ' '):
            value = value[3:]
    return value

with open('ans.csv', 'r', encoding='utf-8') as f:
    csv_reader = csv.reader(f)
    
    next(csv_reader, None) 
    
    for row in csv_reader:
        key = row[0].strip()
        value = myTrim(row[1].strip())
        if (len(value) > 0 and value[0].isalnum()):
            data_dict[key].append(value)
        data_dict_all[key].append(value)

# Solve test currently open in the browser

In [ ]:
def children(elem):
    return elem.find_elements(By.XPATH, './*')

def move_to_next_question():
    time.sleep(0.1)
    button = driver.find_element(By.XPATH, '//*[@name="next"]')
    if button.get_attribute('value') == 'Следующая страница':
        button.click()
        driver.implicitly_wait(4)    
        return True        
    else:
        return False

def solve():
    prompt = driver.find_element(By.XPATH, '//*[@class="qtext"]').text.strip()

    print(prompt)
    answersElem = driver.find_element(By.XPATH, '//*[@class="answer"]')
    childrenElem = answersElem.find_elements(By.XPATH, './*')  

    answers = [myTrim(answer.text.strip()) for answer in childrenElem]


    if prompt in data_dict:
        corrAns = data_dict[prompt]
        
        # Check if there is enough confidence to answer the question
        cnt = 0
        for i in range(len(answers)):
            ans = answers[i]
            
            if ans in corrAns:
                if (cnt != 0):
                    print('multipleAnswersFound')
                cnt+=1  
                if (('*' + ans) in data_dict_all[prompt]):
                    print('incorrectFound')
                    cnt = 5
        if cnt != 1:
            print(data_dict_all[prompt])
            for ans in answers:
                print('correct:',data_dict[prompt].count(ans),'incorrect:', data_dict_all[prompt].count('*' + ans),ans)

        else:
            for i in range(len(answers)):
                ans = answers[i]
                if ans in corrAns:
                    print('found')
                    driver.execute_script("arguments[0].style.color = 'red';", childrenElem[i])
                    children(childrenElem[i])[0].click()
    else:
        print("err")

while True:
    solve()
    if not move_to_next_question():
        break
